# 02 REST API | 03 Json Mode

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_DemoEnvironment/01_Environment.ipynb))

## JSON mode

Azure OpenAI JSON mode is a feature that instructs models from the Azure OpenAI GPT family to return results in JSON format, providing a structured and easily parsable output. This allows developers to seamlessly integrate OpenAI's language models into their applications, facilitating efficient handling and utilization of the generated content to exchange data with other systems or call native functions.


### Further information

[What is JSON mode](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/json-mode)

## Step 1:  Setup Parameters

In [ ]:
#r "nuget: DotNetEnv, 2.5.0"
#r "nuget: System.Text.Json, 7.0.3"
#r "nuget: Newtonsoft.Json, 13.0.1"
using DotNetEnv;

using System.Net;
using System.Net.Http;
using System.Text.Json.Nodes;
using System.Text.Json;

static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);

string apiBase = Environment.GetEnvironmentVariable("SKIT_AOAI1106_ENDPOINT") ?? "SKIT_AOAI1106_ENDPOINT not found";
string apiKey = Environment.GetEnvironmentVariable("SKIT_AOAI1106_APIKEY") ?? "SKIT_AOAI1106_APIKEY not found";
string deploymentName = Environment.GetEnvironmentVariable("SKIT_AOAI1106_DEPLOYMENTNAME") ?? "SKIT_AOAI1106_DEPLOYMENTNAME not found";

string apiVersion = "2023-12-01-preview"; //may change in the future

Expected output
```
Installed Packages
    DotNetEnv, 2.5.0
    Newtonsoft.Json, 13.0.1
    System.Text.Json, 7.0.3
```

## Step 2:  Create completions for chat messages with GPT models

The code cell is using an instance of `HttpClient` to call the REST API of the deployed Azure OpenAI instance.

In [ ]:

var requestPayload = new JsonObject
{
    { "messages", new JsonArray
        {
            new JsonObject
            {
                { "role", "system" },
                { "content", "You are an assistant who helps users to identify the most important keywords from unstructured text. You return results in valid JSON format. You return the recognized keywords as an array with the key 'keywords'" }
                
            },
            new JsonObject
            {
                { "role", "user" },
                { "content", "I need information regarding my order 4711 from last month. My customer number is AE-100-10?" }
            }
        }
    },
    { "response_format", new JsonObject() {
        { "type", "json_object" }
    }},
    { "max_tokens", 200 },
    { "temperature", 0.7 },
    { "frequency_penalty", 0 },
    { "presence_penalty", 0 },
    { "top_p", 0.95 },
    { "stop", null }
};

string payload = JsonSerializer.Serialize(requestPayload, new JsonSerializerOptions
{
    WriteIndented = true // Optional: to make the JSON string more readable
});

        
string endpoint = $"{apiBase}openai/deployments/{deploymentName}/chat/completions?api-version={apiVersion}";

using (HttpClient httpClient = new HttpClient())
{
    httpClient.BaseAddress = new Uri(endpoint);
    httpClient.DefaultRequestHeaders.Add("api-key",apiKey);
    httpClient.DefaultRequestHeaders.Accept.Add(new System.Net.Http.Headers.MediaTypeWithQualityHeaderValue("application/json"));

    var stringContent = new StringContent(payload, Encoding.UTF8, "application/json");

    var response = await httpClient.PostAsync(endpoint, stringContent);

    if (response.IsSuccessStatusCode)
    {
        using (var responseStream = await response.Content.ReadAsStreamAsync())
        {
            // Parse the JSON response using JsonDocument
            using (var jsonDoc = await JsonDocument.ParseAsync(responseStream))
            {
                // Access the message content dynamically
                var root = jsonDoc.RootElement;
                var messageContent = root.GetProperty("choices")[0].GetProperty("message").GetProperty("content").GetString();

                // Output the message content
                Console.WriteLine("Output: " + messageContent);
            }
        }
    }
    else
    {
        Console.WriteLine($"Error: {response}");
    }
}

Expected output:
```
Output: {
  "keywords": ["order 4711", "last month", "customer number", "AE-100-10"]
}
```

## Next Steps

Explore how Azure OpenAI, as one of the first multi modal models, is able to process text and image data as input in the [next notebook](./04_MultiModalVision.ipynb).